# Air quality time series


## Let us first describe the data

This dataset is made of the daily readings of the following air quality values for May 1, 1973 (a Tuesday) to September 30, 1973.

- Ozone: Mean ozone in parts per billion from 1300 to 1500 hours at Roosevelt Island
- Solar.R: Solar radiation in Langleys in the frequency band 4000–7700 Angstroms from 0800 to 1200 hours at Central Park
- Wind: Average wind speed in miles per hour at 0700 and 1000 hours at LaGuardia Airport
- Temp: Maximum daily temperature in degrees Fahrenheit at La Guardia Airport.

** Source: **

The data was obtained from the New York State Department of Conservation (ozone data) and the National Weather Service (meteorological data).

In [ ]:
import pandas as pd
df_temp = pd.read_csv('../data/airquality.csv', 
                      usecols = ["Ozone", "Solar.R", "Wind", "Temp", "Month", "Day"])

# We exclude the first column (= index) because we don't need it.
# To do that, just specify the columns of interest in usecols

#Let's add a year column
df_temp["Year"] = "1973"

# Let's inspect the first few elements
df_temp.head()

In [ ]:
# It would be useful to create a date column (better plotting)
df_temp["Date"] = pd.to_datetime(df_temp["Year"] 
                                 + df_temp["Month"].astype(str) 
                                 + df_temp["Day"].astype(str) , format = "%Y%m%d")

In [ ]:
# Let's inspect the first few elements again
df_temp.head()

In [ ]:
# Check each column's data type
print df_temp.dtypes

# Interpolation

As you can see from the printed values, there are a few NaN in our array.
It probably means that the data was not recorded at this date.

We have several ways of dealing with that :

- We can ignore the relevant data line
- Or we can infer the missing data with, say, an average of a few previous points.

We are going to infer the missing data through interpolation.  
Say we have 1,NaN,2.   
We infer the missing value with $(1+2)/2 = 1.5$

In [ ]:
# Let's see how to do that with pandas

# Get the data to interpolate
ozone = df_temp["Ozone"].values
solar = df_temp["Solar.R"].values
# Get a series of timestamps
timestamps = pd.to_datetime(df_temp["Date"].values)

# Create a new Series with the timestamp as index 
#(or we could have set the index as timestamp in our df_temp dataframe)
s_ozone = pd.Series(ozone, index=timestamps)
s_solar = pd.Series(solar, index=timestamps)

oz_interp = s_ozone.interpolate(method = "time")
sol_interp = s_solar.interpolate(method = "time")

df_temp["Ozone_interp"] = oz_interp.values
df_temp["Solar.R_interp"] = sol_interp.values

df_temp.head(n=10) # use 10 rows to see how interpolation works

In [ ]:
#Let's plot the data
import matplotlib.pylab as plt
from matplotlib.gridspec import GridSpec

%matplotlib inline
fig = plt.figure(figsize=(15, 15))
gs = GridSpec(2, 2, bottom=0.18, left=0.18, right=0.88)

axOz = fig.add_subplot(gs[0])
axSol = fig.add_subplot(gs[1])
axWind = fig.add_subplot(gs[2])
axTemp = fig.add_subplot(gs[3])

# Get time axis
x_date = df_temp["Date"].values

# Get the y values 
y_oz = df_temp["Ozone_interp"].values
y_sol = df_temp["Solar.R_interp"].values
y_temp = df_temp["Temp"].values
y_wind = df_temp["Wind"].values


# Plot
axOz.plot(x_date, y_oz, label = "Ozone in ppm", linewidth=1.5)
axSol.plot(x_date, y_sol, label = "Solar Radiation", linewidth=1.5)
axWind.plot(x_date, y_temp, label = "Temperature in F", linewidth=1.5)
axTemp.plot(x_date, y_wind, label = "Wind", linewidth=1.5)


#####################
# Figure cosmetics
#####################

# Axis labels, legend and formatting
for ax in [axOz, axSol, axWind, axTemp]:
    ax.set_xlabel("time", fontsize=22)
    ax.legend(loc="best", fontsize=22)
    
# improve plot layout
gs.tight_layout(fig, h_pad=3)

#plt.savefig("visual.png") #uncomment to save plot
plt.show() 

# Correlations

We will now look at an interesting way to see interactions between time-series data : running correlations.  

Below is the for the formula of the sample correlation coefficient $r$.

![corr.png](../figures/corr.png)

The running correlation has an extra twist to it. We select a window of points (let's say 30) on which we compute the correlation coefficient. And then we slide this window in time.

So the first value of $r$ is given by points $1, 2, 3, ... 30$.  
The second value of $r$ is given by points $2, 3, 4, ... 31$.
And so on

This is where the phrase "running" comes from. This running correlation gives us information of how the correlation of two time series evolves with time.

In [ ]:
# Clear plotting
plt.clf()
plt.close()

#Let's plot the data
fig = plt.figure(figsize=(15, 15))
gs = GridSpec(2, 1, bottom=0.18, left=0.18, right=0.88)

ax7 = fig.add_subplot(gs[0])
ax30 = fig.add_subplot(gs[1])

# Get time axis
x_date = df_temp["Date"].values

# Get the y values 

r_sol7 = df_temp["Ozone_interp"].rolling(window=7).corr(df_temp["Solar.R_interp"])
r_sol30 = df_temp["Ozone_interp"].rolling(window=30).corr(df_temp["Solar.R_interp"])

r_wind7 = df_temp["Ozone_interp"].rolling(window=7).corr(df_temp["Wind"])
r_wind30 = df_temp["Ozone_interp"].rolling(window=30).corr(df_temp["Wind"])

# On the first few points, no correlation can be computed (because there are not enough previous points)
# This gives a NaN value
# We arbitrarily set this to 0.
r_sol7 = r_sol7.fillna(0)
r_sol30 = r_sol30.fillna(0)
r_wind7 = r_wind7.fillna(0)
r_wind30 = r_wind30.fillna(0)

# Plot
ax7.plot(x_date, r_sol7, label = "7d corr Ozone/Solar", linewidth=1.5, color = "r")
ax7.plot(x_date, r_wind7, label = "7d corr Ozone/Wind", linewidth=1.5, color = "k")
ax30.plot(x_date, r_sol30, label = "30d corr Ozone/Solar", linewidth=1.5, color = "r")
ax30.plot(x_date, r_wind30, label = "30d corr Ozone/Wind", linewidth=1.5, color = "k")

#####################
# Figure cosmetics
#####################

# Axis labels, legend and formatting
for ax in [ax7, ax30]:
    ax.set_xlabel("time", fontsize=22)
    ax.set_ylabel("Correlation coefficient", fontsize=22)
    ax.legend(loc="best", fontsize=22)

# improve plot layout
gs.tight_layout(fig, h_pad=5)

#plt.savefig("visual2.png")
plt.show() #uncomment to plot

# What does this show?

- On a 7 days window, the correlation coefficient changes wildly. 
- There is a single period (july / august) where we see a trend (Solar radiation is correlated to Ozone, Wind is anti-correlated to Ozone)
- On a 30 days window, we see much stronger trends. Wind is strongly anti-correlated to Ozone and the anticorrelation increases up to September. Solar radiation is mildly correlated to Ozone